# How to handle cases where no queries are generated

:::info Prerequisites

This guide assumes familiarity with the following:

- [Query analysis](/docs/tutorials/query_analysis)

:::

Sometimes, a query analysis technique may allow for any number of queries to be generated - including no queries! In this case, our overall chain will need to inspect the result of the query analysis before deciding whether to call the retriever or not.

We will use mock data for this example.

## Setup

### Install dependencies

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/community @langchain/openai zod chromadb
</Npm2Yarn>
```

### Set environment variables

```
OPENAI_API_KEY=your-api-key

# Optional, use LangSmith for best-in-class observability
LANGSMITH_API_KEY=your-api-key
LANGCHAIN_TRACING_V2=true
```

### Create Index

We will create a vectorstore over fake information.

In [ ]:
import { Chroma } from "@langchain/community/vectorstores/chroma"
import { OpenAIEmbeddings } from "@langchain/openai"
import "chromadb";

const texts = ["Harrison worked at Kensho"]
const embeddings = new OpenAIEmbeddings({ model: "text-embedding-3-small" })
const vectorstore = await Chroma.fromTexts(texts, {}, embeddings, {
  collectionName: "harrison"
})
const retriever = vectorstore.asRetriever(1);

## Query analysis

We will use function calling to structure the output. However, we will configure the LLM such that is doesn't NEED to call the function representing a search query (should it decide not to). We will also then use a prompt to do query analysis that explicitly lays when it should and shouldn't make a search.

In [24]:
import { z } from "zod";

const searchSchema = z.object({
    query: z.string().describe("Similarity search query applied to job record."),
});

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />
```

In [25]:
import { zodToJsonSchema } from "zod-to-json-schema";
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { RunnableSequence, RunnablePassthrough } from "@langchain/core/runnables";

const system = `You have the ability to issue search queries to get information to help answer user information.

You do not NEED to look things up. If you don't need to, then just respond normally.`;
const prompt = ChatPromptTemplate.fromMessages(
  [
    ["system", system],
    ["human", "{question}"],
  ]
)
const llmWithTools = llm.bind({
  tools: [{
    type: "function" as const,
    function: {
      name: "search",
      description: "Search over a database of job records.",
      parameters: zodToJsonSchema(searchSchema),
    }
  }]
})
const queryAnalyzer = RunnableSequence.from([
  {
    question: new RunnablePassthrough(),
  },
  prompt,
  llmWithTools
])

We can see that by invoking this we get an message that sometimes - but not always - returns a tool call.

In [26]:
await queryAnalyzer.invoke("where did Harrison work")

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "",
    additional_kwargs: {
      function_call: undefined,
      tool_calls: [
        {
          id: "call_uqHm5OMbXBkmqDr7Xzj8EMmd",
          type: "function",
          function: [Object]
        }
      ]
    }
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "",
  name: undefined,
  additional_kwargs: {
    function_call: undefined,
    tool_calls: [
      {
        id: "call_uqHm5OMbXBkmqDr7Xzj8EMmd",
        type: "function",
        function: { name: "search", arguments: '{"query":"Harrison"}' }
      }
    ]
  }
}

In [27]:
await queryAnalyzer.invoke("hi!")

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "Hello! How can I assist you today?",
    additional_kwargs: { function_call: undefined, tool_calls: undefined }
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "Hello! How can I assist you today?",
  name: undefined,
  additional_kwargs: { function_call: undefined, tool_calls: undefined }
}

## Retrieval with query analysis

So how would we include this in a chain? Let's look at an example below.

In [33]:
import { JsonOutputKeyToolsParser } from "@langchain/core/output_parsers/openai_tools";

const outputParser = new JsonOutputKeyToolsParser({
  keyName: "search",
})

In [46]:
import { RunnableConfig, RunnableLambda } from "@langchain/core/runnables";

const chain = async (question: string, config?: RunnableConfig) => {
  const response = await queryAnalyzer.invoke(question, config);
  if ("tool_calls" in response.additional_kwargs && response.additional_kwargs.tool_calls !== undefined) {
    const query = await outputParser.invoke(response, config);
    return retriever.invoke(query[0].query, config);
  } else {
    return response;
  }
}

const customChain = new RunnableLambda({ func: chain });

In [47]:
await customChain.invoke("where did Harrison Work")

[ Document { pageContent: "Harrison worked at Kensho", metadata: {} } ]

In [48]:
await customChain.invoke("hi!")

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "Hello! How can I assist you today?",
    additional_kwargs: { function_call: undefined, tool_calls: undefined }
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "Hello! How can I assist you today?",
  name: undefined,
  additional_kwargs: { function_call: undefined, tool_calls: undefined }
}

## Next steps

You've now learned some techniques for handling irrelevant questions in query analysis systems.

Next, check out some of the other query analysis guides in this section, like [how to use few-shot examples](/docs/how_to/query_few_shot).